**Importing packages & Uploading files from our github repository**

In [ ]:
!pip install fiona
!pip install geopandas
!pip install statsmodels
!pip install adjustText

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import requests
import io
import fiona
import matplotlib as pm
from sklearn.preprocessing import scale
import folium
import ipywidgets as widgets
from IPython import display
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import statsmodels.api as sm
import geopandas as gpd
import functools as ft
from tabulate import tabulate
from matplotlib.colors import LinearSegmentedColormap
from adjustText import adjust_text
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression

In [ ]:
url1 = 'https://media.githubusercontent.com/media/agrayson-dev/ce190-g1/main/SAC_CITY_TRACT_ZIP_122021.csv'
url2 = 'https://media.githubusercontent.com/media/agrayson-dev/ce190-g1/main/Sac_City_Zips.csv'
url3 = 'https://media.githubusercontent.com/media/agrayson-dev/ce190-g1/main/TTS_LBNL_public_file_07-Sep-2022_all.csv'
url4 = 'https://media.githubusercontent.com/media/agrayson-dev/ce190-g1/main/Zip_Zoning_Intersect.csv'
url5 = 'https://media.githubusercontent.com/media/agrayson-dev/ce190-g1/main/CensusTract_SacramentoEnergyBurden.csv'
url6 = 'https://media.githubusercontent.com/media/agrayson-dev/ce190-g1/main/ces_demog_indicators.csv'
url7 = 'https://media.githubusercontent.com/media/agrayson-dev/ce190-g1/main/ces_enviro_indicators.csv'

tracttozip = pd.read_csv(url1, index_col=0)
saczips = pd.read_csv(url2, index_col=0, header = 0)
saczoning_zip = pd.read_csv(url4, index_col=0)
sac_eb = pd.read_csv(url5, index_col=0)

In [ ]:
sac_eb.dtypes

**Data Cleaning**

In [ ]:
#Sacramento City Zoning by Zipcode
saczoning_zip.rename(columns={'Zip 5':'zip'}, inplace=True)
saczoning_zip = pd.merge(saczoning_zip, tracttozip, on='zip')
saccityzoning = saczoning_zip.groupby(['zip','DESCRIPTION'],as_index=False)[["Area in Square Miles"]].sum()
scz = pd.pivot(saccityzoning, index='zip', columns='DESCRIPTION')[['Area in Square Miles']]
scz = scz.fillna(0)
scz['Total']= scz.sum(axis=1)
scz_perc_full = scz.loc[:,"Agricultural Zone":"Transportation Corridor Zone"].div(scz["Total"], axis=0)*100
scz_perc_full.columns=scz_perc_full.columns.droplevel(0)
scz_perc_full.reset_index(inplace = True)
scz_perc_full.columns
saczips.columns

In [ ]:
scz_perc = scz_perc_full.drop(scz_perc_full.loc[:,'Agricultural Zone':'Central Business District Zone'].columns, axis=1)
scz_perc = scz_perc_full.drop(scz_perc_full.loc[:,'Heavy Industrial Zone':'Manufacturing-Industrial Park Zone'].columns, axis=1)
scz_perc.columns

In [ ]:
#Energy Burden by Zipcode
eb_zip = pd.merge(tracttozip, sac_eb, on='tract')
ebavg_zip = eb_zip.groupby('zip_x')[["Avg. Energy Burden (% income)"]].mean()
ebavg_zip.reset_index(inplace = True)
ebavg_zip.rename(columns={'zip_x':'zip'}, inplace=True)
ebavg_zip.head()

In [ ]:
# Demographic Data (CalEnviroScreen)
ces_demog = pd.read_csv(url6, index_col=0, skiprows=1)
ces_demog.reset_index(inplace = True)
ces_demog.rename(columns={'Census Tract':'tract'}, inplace=True)
ces_demog = pd.merge(tracttozip, ces_demog, on='tract')
ces_envir = pd.read_csv(url7, index_col=0)
ces_envir.rename(columns={'ZIP':'zip'}, inplace=True)
ces_envir = pd.merge(ces_envir, tracttozip, on='zip')
ces_envir = ces_envir.drop(ces_envir.loc[:,'Ozone':'Solid Waste Pctl'].columns, axis=1)
ces_envir = ces_envir.drop(ces_envir.loc[:,'Asthma':'Cardiovascular Disease Pctl'].columns, axis=1)
ces_envir = ces_envir.drop(ces_envir.loc[:,'Pop. Char. ':'tot_ratio'].columns, axis=1)
ces_demog = ces_demog.drop(ces_demog.loc[:,'usps_zip_pref_city':'California County'].columns, axis=1)
ces_demog = (ces_demog.groupby('zip', as_index=False)
       .agg({'Total Population':'sum', 'Children < 10 years (%)':'mean','Pop 10-64 years (%)':'mean','Elderly > 64 years (%)':'mean','Hispanic (%)':'mean','White (%)':'mean','African American (%)':'mean','Native American (%)':'mean','Asian American (%)':'mean','Other/Multiple (%)':'mean'}))
ces_demog.loc[:,'Children < 10 years (%)':'Other/Multiple (%)'] = ces_demog.loc[:,'Children < 10 years (%)':'Other/Multiple (%)'] / 100
ces_demog.columns = ces_demog.columns.str.rstrip('(%)')
ces_envir = (ces_envir.groupby('zip', as_index=False)
       .agg({'CES 4.0 Score':'mean', 'CES 4.0 Percentile':'mean','Pollution Burden':'mean','Pollution Burden Score':'mean','Pollution Burden Pctl':'mean','Education':'mean','Education Pctl':'mean','Linguistic Isolation':'mean','Linguistic Isolation Pctl':'mean','Poverty':'mean','Poverty Pctl':'mean','Unemployment':'mean','Unemployment Pctl':'mean','Housing Burden':'mean','Housing Burden Pctl':'mean'}))
ces_envir.columns=ces_envir.columns.map(lambda x : x+'(%)' if x not in ('zip','CES 4.0 Score', 'CES 4.0 Percentile','Pollution Burden Score','Pollution Burden Pctl','Education Pctl','Linguistic Isolation Pctl','Poverty Pctl','Unemployment Pctl','Housing Burden Pctl')  else x )
ces_envir.head()

In [ ]:
#Solar Data (Tracking the Sun)
tts = pd.read_csv(url3, index_col=0)
tts.rename(columns={'zip_code':'zip'}, inplace=True)
tts = pd.merge(tts, tracttozip, on='zip')
tts.columns

In [ ]:
tts_sac = tts.drop(tts.loc[:,'azimuth_1':'tot_ratio'].columns, axis=1)
zip_solarfreq = tts['zip'].value_counts()
zip_solarfreq = zip_solarfreq.to_frame().reset_index()
zip_solarfreq.rename(columns={'index':'zip','zip':'solarfreq'}, inplace=True)
zip_solarfreq.head(20)
print(tabulate(zip_solarfreq, headers='firstrow', tablefmt='fancy_grid'))

In [ ]:
#merge energy burden, zoning, solar freq, demographic datasets by zip
finaldfs = [zip_solarfreq, ebavg_zip, scz_perc, ces_envir, ces_demog]
finaldata_append = ft.reduce(lambda left, right: pd.merge(left, right, on='zip'), finaldfs)
finaldata_append['total_burden'] = finaldata_append['Housing Burden(%)'] + finaldata_append['Avg. Energy Burden (% income)']
TB = finaldata_append["total_burden"]
solar_zoning = pd.merge(zip_solarfreq, scz_perc, on='zip')
sbdf = [zip_solarfreq, ebavg_zip, ces_envir]
solar_burdens = ft.reduce(lambda left, right: pd.merge(left, right, on='zip'), sbdf)
solar_burdens.insert(1, "total_burden", TB)
solar_demog = pd.merge(zip_solarfreq, ces_demog, on='zip')
finaldata_append.columns

**Generating Background Plots**

In [ ]:
#solar adoption by zip code
plt.figure(figsize=(7,4))
plt.title("Sacramento Solar Adoption by Zipcode")
sns.barplot(x="zip", y="solarfreq", data=zip_solarfreq, color='turquoise');
plt.ylabel("Count of Properties with Rooftop Solar")
plt.tick_params(axis='x', labelrotation = 50)
plt.savefig('solar_count_zip_bar.png')

In [ ]:
#bar plot of energy burden by zip code
plt.figure(figsize=(7,4))
plt.title("Sacramento Energy Burden by Zipcode")
sns.barplot(x="zip", y="Avg. Energy Burden (% income)", data=ebavg_zip, color='green');
plt.tick_params(axis="x", labelrotation = 50)
plt.savefig('energy_burden_zip_bar.png')

In [ ]:
#stacked bar plot for zoning type by zip code
colors = ["#009c3f", "#e526e0", "#16da2d", "#530091", "#6dde5b", "#013cbd", "#99bd00", "#151671", "#ffa226", "#c581ff", "#758200", "#c20075", "#00ac75", "#fc3f00", "#3edae0", "#be0000", "#008566", "#ff5a5c", "#005696",
"#d3c95c", "#4a0440", "#795a00", "#bcb8ff", "#810100", "#1a2d00", "#a90049", "#f8b5b5", "#6a001a", "#ff9490", "#5d343f"]
stac = scz_perc_full.plot(x='zip', kind='bar', stacked=True,
        title='Zone Type Make Up per Zip Code', color=colors)
plt.savefig('zipzone_makeup.png')
box = stac.get_position()
stac.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])
stac.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25),
          fancybox=True, shadow=True, ncol=5)
plt.show()

In [ ]:
#single variate regression analyses - solar (and zip code) as a function of demographic indicators - same as heatmap outputs, different method
standardisedX = scale(finaldata_append)
standardisedX = pd.DataFrame(standardisedX, index=finaldata_append.index, columns=finaldata_append.columns)
X = standardisedX[['White ']]
X1 = standardisedX[['African American ']]
X2 = standardisedX[['Native American ']]
X3 = standardisedX[['Hispanic ']]
X4 = standardisedX[['Asian American ']]
X5 = standardisedX[['Other/Multiple ']]
X6 = standardisedX[['Children < 10 years ']]
X7 = standardisedX[['Pop 10-64 years ']]
X8 = standardisedX[['Elderly > 64 years ']]
y = standardisedX[['solarfreq']]
y1 = standardisedX[['zip']]
ols_reg_white = sm.OLS(y, X).fit()
ols_reg_aa = sm.OLS(y, X1).fit()
ols_reg_ind = sm.OLS(y, X2).fit()
ols_reg_hisp = sm.OLS(y, X3).fit()
ols_reg_asian = sm.OLS(y, X4).fit()
ols_reg_other = sm.OLS(y, X5).fit()
ols_reg_children = sm.OLS(y, X6).fit()
ols_reg_mid = sm.OLS(y, X7).fit()
ols_reg_elder = sm.OLS(y, X8).fit()
ols_reg_whitezip = sm.OLS(y1, X).fit()
ols_reg_aazip = sm.OLS(y1, X1).fit()
ols_reg_indzip = sm.OLS(y1, X2).fit()
ols_reg_hispzip = sm.OLS(y1, X3).fit()
ols_reg_asianzip = sm.OLS(y1, X4).fit()
ols_reg_otherzip = sm.OLS(y1, X5).fit()
print(ols_reg_elder.summary())
plt.rc('figure', figsize=(12, 7))
plt.text(0.01, 0.05, str(ols_reg_white.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
plt.axis('off')
plt.tight_layout()
plt.savefig('ols_reg_white.png')

In [ ]:
#unpack the demographics of the one zip code with insanely high adoption
plt.text(15+0.2, 22, "95831", horizontalalignment='left', size='medium', color='black', weight='semibold')
sns.scatterplot(
x='total_burden',
y='Poverty(%)',
hue = 'solarfreq',
data=finaldata_append)

In [ ]:
#correlation plots
corr1 = solar_burdens.corr()
plt.figure(figsize=(16,5))
sns.heatmap(corr1, cmap="Reds", annot=True);
plt.tick_params(axis='x')
plt.savefig('correlation_plot.png', bbox_inches='tight')

In [ ]:
corr2 = solar_zoning.corr()
plt.figure(figsize=(16,5))
sns.heatmap(corr2, cmap="Reds", annot=True);
plt.savefig('solar_zoning_corrl.png', bbox_inches='tight')

**Cal Enviro Screen**

Environmental Indicators

In [ ]:
 %%html
 <iframe src="https://drive.google.com/file/d/1XlDhkMnHqfdkGHZBsZgoDRBKOhcYul41/preview" width="640" height="480" allow="autoplay"></iframe>

Demographic Indicators

In [ ]:
%%html
<iframe src="https://drive.google.com/file/d/1KFj-6VO3LKZosNwAb3H8aFzj5i5WvLp4/preview" width="640" height="480" allow="autoplay"></iframe>

**Geospatial Analysis**

In [ ]:
#Loading in Zip Zoning shapefile from OpenData Sacramento
url = 'https://github.com/agrayson-dev/ce190-g1/blob/main/Intersect_of_Zip_Codes_and_Zoning%20(1).zip?raw=true'
request = requests.get(url)
b = bytes(request.content)
with fiona.BytesCollection(b) as f:
    crs = f.crs
    sac_shp = gpd.GeoDataFrame.from_features(f, crs=crs)
sac_shp.dtypes

In [ ]:
#Merge CSV data to shapefile
sac_shp.rename(columns={'ZIP5':'zip'}, inplace=True)
finaldata_append['zip'] = finaldata_append['zip'].apply(lambda x: str(x))
final_shp = finaldata_append.merge(sac_shp, on='zip', how='left')
#final_shp.head()
final_shp = gpd.GeoDataFrame(final_shp, geometry='geometry')
final_shp.plot();

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

#map % MFH by zip code
final_shp.plot(column="Multi-Unit Dwelling Zone", ax=ax, legend = True, cax=cax, cmap = 'tab20');
ax.set_title('MFH by zip code')
plt.show()
plt.savefig('MFH_zipmap.png', bbox_inches='tight')


In [ ]:
#map % Energy Burden by zip code
fig, ax = plt.subplots(figsize = (10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

final_shp.plot(column="Avg. Energy Burden (% income)", ax=ax, legend = True, cax=cax, cmap = 'coolwarm');
ax.set_title('Energy Burden by Zip Code (% income)')
plt.savefig('EB_zipmap.png', bbox_inches='tight')
plt.show()

In [ ]:
#map Solar Frequency by zip code
df=final_shp
df["solar_percap"]=df["solarfreq"]/df["Total Population"]
print(df)
fig, ax = plt.subplots(figsize = (10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

final_shp.plot(column="solar_percap", ax=ax, legend = True, cax=cax, cmap = 'coolwarm');
ax.set_title('Count of Properties with Rooftop Solar Per Capita')
plt.savefig('SF_zipmap.png', bbox_inches='tight')
plt.show()